In [1]:
import re
import numpy as np
import datetime
import time

from apiclient.discovery import build

from tqdm import tqdm_notebook as tqdm
from ipywidgets import IntProgress

In [2]:
API_KEY='ここにAPIKEYを入れる'

In [3]:
id_ = 'UCXcjvt8cOfwtcqaMeE7-hqA' #SUSURUTVID

In [4]:
def YoutubeChannelDetails(id_, API_KEY):
    API_SERVICE_NAME = "youtube"
    API_VERSION = "v3"

    youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    search_response = youtube.channels().list(
    part= 'snippet,contentDetails',
    id=id_,
    ).execute()
    
    return search_response['items'][0]

In [5]:
ChannelDetails = YoutubeChannelDetails(id_,API_KEY)

In [6]:
ChannelTitle = '-'.join(ChannelDetails['snippet']['title'].split(' '))
uploads = ChannelDetails['contentDetails']['relatedPlaylists']['uploads']

In [7]:

def YoutubePlaylistContents(id_, API_KEY):
    
    responses = []
    nextPageToken = 'start'
    counts = 0

    while(nextPageToken is not None):
        
        API_SERVICE_NAME = "youtube"
        API_VERSION = "v3"

        youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

        if(nextPageToken == 'start'):
            search_response = youtube.playlistItems().list(
            part= 'snippet',
            playlistId=id_,
            maxResults = 50,
            ).execute()
            nextPageToken = search_response['nextPageToken']
        else:
            search_response = youtube.playlistItems().list(
            part= 'snippet',
            playlistId=id_,
            maxResults = 50,
            pageToken = nextPageToken
            ).execute()
            try:
                nextPageToken = search_response['nextPageToken']
            except:
                nextPageToken = None
        
        responses.extend(search_response['items'])
        counts += len(search_response['items'])
    
    print('load '+str(counts)+' videos...')
    
    return responses

In [8]:
total_contents = YoutubePlaylistContents(uploads,API_KEY)

load 2116 videos...


In [9]:
dic_total = []
for t in total_contents:
    
    date_list = t['snippet']['publishedAt'].split('T')
    year, month, date = date_list[0].split('-')
    hour, minute, sec = date_list[1].split(':')
    sec = sec[:2]
    
    dic = {'title':t['snippet']['title'], 
           'Id':t['snippet']['resourceId']['videoId'],
           'description':t['snippet']['description']}
    
    dic_total.append(dic)

In [10]:
def ConvertDuration(string):
    string = string.replace('PT', '') 
    strings = re.split('\D',string)[:-1]
    if(len(strings) == 3):
        delta = datetime.timedelta(hours=int(strings[0]),
                                   minutes=int(strings[1]),
                                   seconds=int(strings[2]))
    elif(len(strings) == 2):
        delta = datetime.timedelta(minutes=int(strings[0]),
                                   seconds=int(strings[1]))
    elif(len(strings) == 1):
        delta =datetime.timedelta(seconds=int(strings[0]))
    else:
        delta = datetime.timedelta(seconds=0)
    
    return delta.seconds

def YoutubeVideoDetails(id_, API_KEY):
    API_SERVICE_NAME = "youtube"
    API_VERSION = "v3"

    youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    search_response = youtube.videos().list(
    part= 'statistics,contentDetails',
    id=id_,
    ).execute()
    
    hoge = search_response['items'][0]
    details = {'viewCount':int(hoge['statistics']['viewCount']),
               'likeCount':int(hoge['statistics']['likeCount']),
               'dislikeCount':int(hoge['statistics']['dislikeCount']),
               'commentCount':int(hoge['statistics']['commentCount']),
               'duration':ConvertDuration(str(hoge['contentDetails']['duration']))}
    
    return details

In [11]:
for n,d in enumerate(tqdm(np.array([i['Id'] for i in dic_total]))):
    details = YoutubeVideoDetails(d,API_KEY)
    dic_total[n].update(details)
dic_total = np.array(dic_total)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/2116 [00:00<?, ?it/s]

KeyError: 'commentCount'

In [13]:
dic_total[2115]

{'title': 'SUSURU TV.第0回放送\u3000衝撃の展開！！！',
 'Id': '-d8K7yNM0wU',
 'description': 'ラーメンyoutuberSUSURUによる\n「毎日ラーメン健康生活」をお届け。\n毎日ラーメンをすすります。\n\nチャンネル登録\nhttps://www.youtube.com/channel/UCXcjvt8cOfwtcqaMeE7-hqA?sub_confirmation=1\n\ntwitter\nhttps://twitter.com/susuru_tv\nFacebookページ\nhttps://www.facebook.com/Susuru-1636122563334708/'}

In [14]:
dic_total[1]

{'title': '起きてすぐ家系ラーメンをすすりたい日があったんですよ。をすする 麺家ばく【飯テロ】SUSURU TV.第1935回',
 'date': datetime.datetime(2021, 2, 21, 9, 30, 2),
 'Id': 'YMnQ3eXwxhQ',
 'description': '★SUSURU TV.の公式グッズサイトができました！★\nhttps://susurutv.stores.jp/\n\n■お仕事の依頼は■\ninfo@susurulab.co.jp\n企業様のお問い合わせは上記よりお願いいたします。\n\n▼SUSURUのサブチャンネルもよろしく！▼\nhttps://www.youtube.com/channel/UCVdjYnmDsyiUcpcryI-k6ZA\n\nご視聴ありがとうございます。\nラーメンを毎日すする「毎日ラーメン健康生活」を実施中！\nチャンネル登録もよろしくお願いします！ \n\n【本日のラーメン情報】\n麺家ばく\n東京都中野区東中野4-4-3 山内ビル 1F\nhttps://tabelog.com/tokyo/A1319/A131901/13049677/\n\n■twitter\nhttps://twitter.com/susuru_tv\n\n■instagram\nhttps://www.instagram.com/susuru_tv/\n\n\n★メンバーシップも始めました★\nhttps://www.youtube.com/channel/UCXcjvt8cOfwtcqaMeE7-hqA/join\n\n★SUSURU TV.新曲配信中！★\nhttps://linkco.re/u5VA13XM\n\n■LINEスタンプ\nhttps://store.line.me/stickershop/product/1405507/ja\n\nもしくはスタンプショップで「susuru」と検索してください！\n\n■SUSURUのグッズ\nhttps://p-town.dmm.com/entertainers/23\n※パーカーもVV限定で発売しています！\n\n◇1000万再生ごとの感謝記念MV◇\nhttps://www.youtube.com/playlist?list=PLR

In [14]:
import pickle

In [15]:
with open ("NEWSUSURU.pickle",mode="wb")as f:
    pickle.dump(dic_total,f)